In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spotify-top-2000s-mega-dataset/Spotify-2000.csv


In [ ]:
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('/kaggle/input/spotify-top-2000s-mega-dataset/Spotify-2000.csv')

In [ ]:
df.head(4)

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4,76


In [ ]:
df.tail(4)

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
1990,1991,Hound Dog,Elvis Presley,adult standards,1958,175,76,36,-8,76,95,136,73,6,69
1991,1992,Johnny B. Goode,Chuck Berry,blues rock,1959,168,80,53,-9,31,97,162,74,7,74
1992,1993,Take Five,The Dave Brubeck Quartet,bebop,1959,174,26,45,-13,7,60,324,54,4,65
1993,1994,Blueberry Hill,Fats Domino,adult standards,1959,133,50,49,-10,16,83,148,74,3,56


In [ ]:
df.shape

(1994, 15)

In [ ]:
df.isnull().sum()

Index                     0
Title                     0
Artist                    0
Top Genre                 0
Year                      0
Beats Per Minute (BPM)    0
Energy                    0
Danceability              0
Loudness (dB)             0
Liveness                  0
Valence                   0
Length (Duration)         0
Acousticness              0
Speechiness               0
Popularity                0
dtype: int64

In [ ]:
df = df[['Title', 'Top Genre']]
df.head

<bound method NDFrame.head of                        Title            Top Genre
0                    Sunrise      adult standards
1                Black Night           album rock
2             Clint Eastwood  alternative hip hop
3              The Pretender    alternative metal
4     Waitin' On A Sunny Day         classic rock
...                      ...                  ...
1989        Heartbreak Hotel      adult standards
1990               Hound Dog      adult standards
1991         Johnny B. Goode           blues rock
1992               Take Five                bebop
1993          Blueberry Hill      adult standards

[1994 rows x 2 columns]>

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(1994, 2)

In [ ]:
df.head(4)

,Title,Top Genre
0,Sunrise,adult standards
1,Black Night,album rock
2,Clint Eastwood,alternative hip hop
3,The Pretender,alternative metal


In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(df['Top Genre'])

# Transforming genre labels into encoded values
df['Encoded Genre'] = label_encoder.transform(df['Top Genre'])

In [ ]:
def recommendation(song_title, n=11):
    song_genre = df.loc[df['Title'] == song_title, 'Top Genre'].values[0]
    genre_songs = df[df['Top Genre'] == song_genre]
    if len(genre_songs) < n:
            n = len(genre_songs)  # Limit n to the available songs
            print(f"There are only {n} songs in the '{song_genre}' genre.")

    recommended_songs = genre_songs.sample(n)['Title'].tolist()
    return song_genre, recommended_songs

input_song = 'Take Five'
song_genre, recommended_songs = recommendation(input_song)
print("Genre based on the input song:", song_genre)
print("Recommended songs from the same genre:")
print(recommended_songs)


There are only 1 songs in the 'bebop' genre.
Genre based on the input song: bebop
Recommended songs from the same genre:
['Take Five']


In [ ]:
def recommendation(song_title, n=11):
    song_genre = df.loc[df['Title'] == song_title, 'Top Genre'].values[0]
    genre_songs = df[df['Top Genre'] == song_genre]
    if len(genre_songs) < n:
            n = len(genre_songs)  # Limit n to the available songs
            print(f"There are only {n} songs in the '{song_genre}' genre.")

    recommended_songs = genre_songs.sample(n)['Title'].tolist()
    return song_genre, recommended_songs

input_song = 'Johnny B. Goode'
song_genre, recommended_songs = recommendation(input_song)
print("Genre:", song_genre)
print("Recommended songs:")
print(recommended_songs)

Genre: blues rock
Recommended songs:
["Europa (Earth's Cry Heaven's Smile)", 'LA Song', 'Me and Bobby McGee', 'Samba Pa Ti', 'Cry Baby', 'Mercedes Benz', "She's Not There", 'Smooth (feat. Rob Thomas)', 'Bird of Paradise', 'Leave the Light On', 'Johnny B. Goode']


In [ ]:
import pickle
pickle.dump(df,open('similer.pkl','wb'))
pickle.dump(df,open('mu.pkl','wb'))